## Let's Make a Journal-O-Mancy! ##

In this era of self-quarantining, I've been good at keeping up my journals. What better way to ensure that I will be keeping up my habit than creating a fun journal-o-mancy that I can use after each journal entry?

### Tools to be Used ###

1. First, I'm going to put in my journal entry, and do a sentiment analysis with [TextBlob](https://textblob.readthedocs.io/en/dev/). Then, using TextBlob's corpus, a list of keywords will be generated that has a similar sentiment score.

2. Then, using the keyword(s), I'm hoping to use the [Advice Slip JSON API](https://api.adviceslip.com/#object-slip) to get a corresponding advice.

3. Last, using the same keyword(s), I'm hoping to use [SearchLY API](https://searchly.asuarez.dev/docs/v1), which can suggest songs by lyric content similarity.

In [21]:
#Let's install TextBlob and the corpus!
#Skip if you already have this
!pip3 install -U textblob
!python -m textblob.download_corpora

In [22]:
from textblob import TextBlob

In [23]:
journal_entry = open("sample_entry.txt","r") 

entry = TextBlob(str(journal_entry.readlines()))

journal_entry.close() 


You can do fun things with TextBlob, such as extracting the noun phrases like this:

In [56]:
entry.noun_phrases

##We'll use this later!

WordList(['dear diary', 'laika', 'super hyper', 'who', 'peanut butter', 'flop flop flop whenever', 'difficult times'])

### Let's do a Sentiment Analysis! ###

From TextBlob about the `.sentiment` property:
> The sentiment property returns a namedtuple of the form Sentiment(polarity, subjectivity). The polarity score is a float within the range [-1.0, 1.0]. The subjectivity is a float within the range [0.0, 1.0] where 0.0 is very objective and 1.0 is very subjective.


In [25]:
print(entry.sentiment)

Sentiment(polarity=0.3048888888888889, subjectivity=0.5665555555555555)


What does this mean?
Polarity is 0.75, which means that the statement is positive and 0.75 subjectivity refers that it is an opinion rather than fact.

In [26]:
## This should be included in your Python packages
import xml.etree.ElementTree as ET
tree = ET.parse('en-sentiment.xml')

In [34]:
polarity_score = str(entry.sentiment.polarity)
subjectivity_score = str(entry.sentiment.subjectivity)
error_margin = 0.1 ##needs to be somewhere between 1 and 0

wordlist = []
for element in tree.iterfind('word'):
    if str(float(polarity_score)+ error_margin) > element.attrib['polarity'] > str(float(polarity_score)-error_margin):
            if str(float(subjectivity_score)+ error_margin) > element.attrib['subjectivity'] > str(float(subjectivity_score)-error_margin):
                wordlist.append(element.attrib['form'])
print (wordlist)

['accurate', 'accurate', 'advanced', 'applicative', 'apportioned', 'bizarre', 'come-at-able', 'compelling', 'complimentary', 'detailed', 'ecological', 'enjoy', 'for sure', 'fresh', 'genuine', 'guarded', 'high', 'incomparable', 'lifelike', 'lonely', 'naturalistic', 'nonviolent', 'risk-free', 'secure', 'smooth', 'straight', 'well-off']


## Advice Slip API ##
Let's try to get advice with these keywords!

In [35]:
import requests

[Dataquest.io]('https://www.dataquest.io/blog/python-api-tutorial') was very helpful in understanding API requests. 

Here's a few key API Status Codes:

>200: Everything went okay, and the result has been returned (if any).

>404: The resource you tried to access wasn’t found on the server.

Let's check our API is working!

In [29]:
#using 'love' as a dummy keyword, let's get the API.
response = requests.get("https://api.adviceslip.com/advice/search/love")

# Hopefully this is 200!
print(response.status_code)

#Let's print out what we get back in JSON
print(response.json())

200
{'total_results': '5', 'query': '', 'slips': [{'advice': "Alway do anything for love, but don't do that.", 'slip_id': '101'}, {'advice': 'Be a good lover.', 'slip_id': '174'}, {'advice': 'Never waste an opportunity to tell someone you love them.', 'slip_id': '202'}, {'advice': "You can fail at what you don't want. So you might as well take a chance on doing what you love.", 'slip_id': '184'}, {'advice': "You can fail at what you don't want. So you might as well take a chance on doing what you love.", 'slip_id': '185'}]}


Let's do this for real!

In [46]:
max_tries = len(wordlist)
current_tries = 0
master_advice = []

print(("max tries # is ") + str(max_tries))

def get_advice():
    global current_tries
    search = wordlist[current_tries]
    response = requests.get("https://api.adviceslip.com/advice/search/"+ search)
    try: 
        if response.json()['total_results'][0]:
            advice = response.json()['slips'][0]['advice']
            master_advice.append(advice)
            print("found advice for "+ str(wordlist[current_tries]))
    except KeyError:
        print((wordlist[current_tries])+(": didn't work."))
        if current_tries == max_tries:
            print("No advice found with keyword!")
        current_tries += 1
#         print(current_tries)
        get_advice()
        
## Let's give this function a spin
get_advice()

max tries # is 27
accurate: didn't work.
accurate: didn't work.
advanced: didn't work.
applicative: didn't work.
apportioned: didn't work.
bizarre: didn't work.
come-at-able: didn't work.
compelling: didn't work.
complimentary: didn't work.
detailed: didn't work.
ecological: didn't work.
found advice for enjoy


Let's try to get advice!

In [47]:
print(master_advice[0])

Enjoy a little nonsense now and then.


## SearchLY API ##
Let's try to get songs with these keywords!

In [48]:
#Let's grab the nouns from before!!
punctuation = ".,''[]"  # add whatever you want

noun_list = str(entry.noun_phrases)
for c in noun_list:
    if c in punctuation:
        noun_list = noun_list.replace(c, "")

print(noun_list)


dear diary laika super hyper who peanut butter flop flop flop whenever difficult times


In [49]:
url = 'https://searchly.asuarez.dev/api/v1/similarity/by_content'
request_body = {"content": noun_list}

# Make the classify request
# We only need the first JSON response!
response = requests.post(url, json=request_body).json()['response']['similarity_list'][0]

In [53]:
print(response)

{'artist_name': 'mc chris', 'artist_url': 'https://www.azlyrics.com/m/mcchris.html', 'id': 36769, 'index_id': 36765, 'lyrics': "your rapper, he's wack dude, but does he even try?, can he do what mine do? think you should say buh-bye. get up on the mic like a five on a fifty. quickly avoid the hickeys from the bucktoothed bitties. fake timberlake just to be by britney. smoke that pipe with whitney, shoot that blow with iggy. l.i.b, n.y.c. and all places in between, you could call me a mint cuz i make the green. i make the scene, i make believe that you all was naked, so i wouldn't have to fake it, just copy and paste it, like adobe photoshop, red foreman in robocop, i get up on the mic and you know i won't fuckin stop. it's like the props of carrot top, or yellow stains in my socks. you acting like you hip? you all hepped up on hops!, yo let's do the body rock, cuz the beat just be so bumpin, let's get our groove on before our carriage is a pumpkin, before they outlaw fuckin, not bad fo

In [57]:
artist = response['artist_name']
song = response['song_name']
lyrics = response['lyrics']
url = response['song_url']

print("Artist: " + artist)
print("Song: " + song)
print("Lyrics: " + lyrics)
print("URL: " + url)

Artist: mc chris
Song: dq blizzard
Lyrics: your rapper, he's wack dude, but does he even try?, can he do what mine do? think you should say buh-bye. get up on the mic like a five on a fifty. quickly avoid the hickeys from the bucktoothed bitties. fake timberlake just to be by britney. smoke that pipe with whitney, shoot that blow with iggy. l.i.b, n.y.c. and all places in between, you could call me a mint cuz i make the green. i make the scene, i make believe that you all was naked, so i wouldn't have to fake it, just copy and paste it, like adobe photoshop, red foreman in robocop, i get up on the mic and you know i won't fuckin stop. it's like the props of carrot top, or yellow stains in my socks. you acting like you hip? you all hepped up on hops!, yo let's do the body rock, cuz the beat just be so bumpin, let's get our groove on before our carriage is a pumpkin, before they outlaw fuckin, not bad for a drunken munchkin. my name is mc chris welcome to my lyric luncheon. word up, word

## Let's put it all Together! ##

In [55]:
print("Journal Entry")
print(entry)
print(" ")
print("Today's Advice for You:")
print(master_advice[0])
print(" ")
print("Today's Theme Song:")
print("Artist: " + artist)
print("Song: " + song)
print("URL: " + url)

Journal Entry
["Dear Diary, my dog Laika has been super hyper during this pandemic. Who know's what he's been up to? While I am trying my best to do homework, he keeps running around the house looking for peanut butter. I love him so much, but I do wish he would calm down! His ears are very cute and goes flop flop flop whenever we go out for a walk. He keeps me distracted during these difficult times, and for that I love him very much. "]
 
Today's Advice for You:
Enjoy a little nonsense now and then.
 
Today's Theme Song:
Artist: mc chris
Song: dq blizzard
URL: https://www.azlyrics.com/lyrics/mcchris/dqblizzard.html
